In [ ]:
import pandas as pd


df = pd.read_parquet('../base.data/employees.parquet', engine='pyarrow')

# print(df)

df = pd.read_parquet('../base.data/customers.parquet', engine='pyarrow')

print(df)


In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import wkb


df = pd.read_parquet('../base.data/50001.parquet', engine='pyarrow')

print(df)

# Convertir la columna de geometría a un objeto de geometría
df['geometry'] = df['geometry'].apply(lambda x: wkb.loads(x, hex=True))

# Convertir el DataFrame de pandas a un GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Convertir el GeoDataFrame a GeoJSON
medellin = gdf.to_json()

print(medellin)

with open('medellin.json', 'w') as f:
    f.write(medellin)

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import wkb

df = pd.read_parquet('../base.data/medellin_neighborhoods.parquet', engine='pyarrow')

# print(df)

# Convertir la columna de geometría a un objeto de geometría
df['geometry'] = df['geometry'].apply(lambda x: wkb.loads(x, hex=True))

# Convertir el DataFrame de pandas a un GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Convertir el GeoDataFrame a GeoJSON
barrios = gdf.to_json()

print(barrios)

with open('barrios.json', 'w') as f:
    f.write(barrios)

In [ ]:
import folium
from turfpy.measurement import centroid, point_on_feature
from geojson import Feature
import json

# Cargar el GeoJSON
with open('medellin.json', 'r') as f:
    medellin = json.load(f)
    
# Cargar el GeoJSON
with open('barrios.json', 'r') as f:
    barrios = json.load(f)

center_point = centroid(Feature(geometry=medellin["features"][0]["geometry"]))

# Crear un mapa base
m = folium.Map(location=[center_point["geometry"]["coordinates"][1], center_point["geometry"]["coordinates"][0]], zoom_start=12)

# Agregar el GeoJSON al mapa
# folium.GeoJson(medellin).add_to(m)
# folium.GeoJson(barrios).add_to(m)


# Agregar el GeoJSON al mapa con popups
folium.GeoJson(
    barrios,
    tooltip=folium.GeoJsonTooltip(fields=["CODIGO", "NOMBRE", "IDENTIFICACION", "LIMITEMUNICIPIOID", "SUBTIPO_COMUNACORREGIMIENTO"],
                                  aliases=["CODIGO:", "NOMBRE:", "IDENTIFICACION:", "LIMITEMUNICIPIOID:", "SUBTIPO_COMUNACORREGIMIENTO:"])
).add_to(m)

m
    

In [ ]:
from turfpy.measurement import bbox
from geojson import Polygon
box = bbox(medellin["features"][0]["geometry"])

print(box)


In [ ]:
from geojson import Point, MultiPolygon
from turfpy.measurement import boolean_point_in_polygon
import random

# Definir las coordenadas aproximadas de Medellín
lat_range = (box[1], box[3])  # Latitud de Medellín
lon_range = (box[0], box[2])  # Longitud de Medellín

lat_rand = round(random.uniform(*lat_range), 6)
lng_rand = round(random.uniform(*lon_range), 6)


point = Feature(geometry=Point([lng_rand, lat_rand]))

polygons = [feature["geometry"]["coordinates"] for feature in barrios["features"]]

# Crear un MultiPolygon
multi_polygon = MultiPolygon(polygons)

boolean_point_in_polygon(point, multi_polygon)

In [4]:
import json
import time
import socket
import uuid
import random
from datetime import datetime
import pytz
from turfpy.measurement import bbox, boolean_point_in_polygon
from geojson import Feature, Point, Polygon, MultiPolygon

# Cargar el GeoJSON de Medellín
with open('medellin.json', 'r') as f:
    medellin = json.load(f)
    
# Cargar el GeoJSON de las comunas y corregimientos
with open('barrios.json', 'r') as f:
    barrios = json.load(f)

box = bbox(medellin["features"][0]["geometry"])

# Definir las coordenadas aproximadas de Medellín
lat_range = (box[1], box[3])  # Latitud de Medellín
lng_range = (box[0], box[2])  # Longitud de Medellín


# Generar un punto aleatorio
def generate_point():
    while(True):
        lat_rand = round(random.uniform(*lat_range), 6)
        lng_rand = round(random.uniform(*lng_range), 6)
        point = Feature(geometry=Point([lng_rand, lat_rand]))
        polygons = [feature["geometry"]["coordinates"] for feature in barrios["features"]]
        multi_polygon = MultiPolygon(polygons)
        if boolean_point_in_polygon(point, multi_polygon):
            return lat_rand, lng_rand

# Generar datos simulados
def generate_event():
    lat, lng = generate_point()
    event = {
        "latitude": lat,
        "longitude": lng,
        "date": datetime.now(pytz.timezone('America/Bogota')).strftime("%d/%m/%Y %H:%M:%S"),
        "customer_id": random.randint(1000, 2000),
        "employee_id": random.randint(9000, 9040),
        "quantity_products": random.randint(10, 50),
        "order_id": str(uuid.uuid4()),
        # "commune": random.choice(comunas),
        # "neighborhood": random.choice(barrios),
        # "partition_date": datetime.now().strftime("%d%m%Y"),
        # "event_date": datetime.now().strftime("%d/%m/%Y %H:%M:%S"),
        # "event_day": datetime.now().day,
        # "event_hour": datetime.now().hour,
        # "event_minute": datetime.now().minute,
        # "event_month": datetime.now().month,
        # "event_second": datetime.now().second,
        # "event_year": datetime.now().year
    }
    return event

def send_data(s, conn, data):
    conn.sendall(json.dumps(data).encode() + b'\n')
    print("enviado:", data)
    

def main():
    
    host = 'localhost'  # Nombre del host o dirección IP
    port = 4229  # Puerto en el que escucha el servidor

    s = socket.socket()
    s.bind((host, port))
    s.listen(1)
    print(f"Listening on {host}:{port}")

    conn, addr = s.accept()
    print(f"Connection from {addr}")
    try:
        while True:
            event = generate_event()
            # Esperar 30 segundos antes de generar el siguiente evento
            send_data(s, conn, event)
            # Ejemplo de guardado en un archivo JSON
            with open('simulated_data.json', 'a') as file:
                file.write(json.dumps(event) + "\n")
            
            time.sleep(1)
            
    except KeyboardInterrupt:
        print("Deteniendo el servidor...")
        conn.close()
        s.close()
        print("Conexión cerrada.")
    
    except BrokenPipeError:
        print("La conexión ha sido cerrada.")
        conn.close()
        s.close()


if __name__ == "__main__":
    main()

Listening on localhost:4229
Connection from ('127.0.0.1', 38060)
enviado: {'latitude': 6.297579, 'longitude': -75.549243, 'date': '12/06/2024 13:32:51', 'customer_id': 1762, 'employee_id': 9018, 'quantity_products': 47, 'order_id': 'dd8d9d9f-1de2-4411-8895-bbbd9daf7c36'}
enviado: {'latitude': 6.244971, 'longitude': -75.586698, 'date': '12/06/2024 13:32:53', 'customer_id': 1266, 'employee_id': 9018, 'quantity_products': 26, 'order_id': '8814a751-c73e-4582-838c-090f2c0aca58'}
enviado: {'latitude': 6.189465, 'longitude': -75.578689, 'date': '12/06/2024 13:32:56', 'customer_id': 1172, 'employee_id': 9031, 'quantity_products': 27, 'order_id': '3e8ae64a-bbe6-4e97-8d78-cb39f99ef255'}
enviado: {'latitude': 6.275753, 'longitude': -75.523645, 'date': '12/06/2024 13:32:57', 'customer_id': 1578, 'employee_id': 9026, 'quantity_products': 24, 'order_id': 'c00b9a3f-caa7-487a-9ee1-b594d5e6c811'}
enviado: {'latitude': 6.266872, 'longitude': -75.565157, 'date': '12/06/2024 13:32:59', 'customer_id': 1108

In [2]:
import json
import pandas as pd
import folium
from folium.plugins import HeatMap

# Leer el archivo JSON línea por línea y cargar los datos en un DataFrame de pandas
data = []
with open('simulated_data.json', 'r') as file:
    for line in file:
        data.append(json.loads(line))

df = pd.DataFrame(data)

# Crear una lista de puntos ponderados por quantity_products
heat_data = [[row['latitude'], row['longitude'], row['quantity_products']] for index, row in df.iterrows()]

# Crear un mapa de Folium centrado en una ubicación inicial (puedes ajustar la ubicación y el zoom)
m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=12)

# Agregar el HeatMap al mapa
HeatMap(heat_data).add_to(m)

# Guardar el mapa en un archivo HTML
# m.save('heatmap.html')

# Mostrar el mapa en un notebook (opcional, solo si estás usando Jupyter Notebook)
m